In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install streamlit

## Tạo app.py

In [ ]:
%%writefile /content/drive/MyDrive/CODE/KHKT/app.py
import streamlit as st
import numpy as np
import joblib
from PIL import Image
from tempfile import NamedTemporaryFile
import requests
import sys
import os
import math
from io import BytesIO
from difflib import get_close_matches
import unicodedata

# ADD MODULE PATH (để import extract_features.py)
sys.path.append("/content/drive/MyDrive/CODE/KHKT")

# Import segmentation model & feature extractor
from extract_features import extract_features, predict_flood_level

# Load future flood predictor model
predictor_model = joblib.load("/content/drive/MyDrive/CODE/KHKT/future_flood_multistep.pkl")

# WEATHER API KEY
API_KEY = "b3e86069b418794e0effd1c9d29c2466"

# 1. DANH SÁCH TỈNH THÀNH VIỆT NAM + GỢI Ý
VIETNAM_PROVINCES = [
    "An Giang", "Ba Ria - Vung Tau", "Bac Giang", "Bac Kan", "Bac Lieu", "Bac Ninh",
    "Ben Tre", "Binh Dinh", "Binh Duong", "Binh Phuoc", "Binh Thuan",
    "Ca Mau", "Can Tho", "Cao Bang",
    "Da Nang", "Dak Lak", "Dak Nong", "Dien Bien", "Dong Nai", "Dong Thap",
    "Gia Lai", "Ha Giang", "Ha Nam", "Ha Noi", "Ha Tinh",
    "Hai Duong", "Hai Phong", "Hau Giang", "Hoa Binh",
    "Ho Chi Minh City", "HCM",
    "Hung Yen", "Khanh Hoa", "Kien Giang", "Kon Tum",
    "Lai Chau", "Lam Dong", "Lang Son", "Lao Cai", "Long An",
    "Nam Dinh", "Nghe An", "Ninh Binh", "Ninh Thuan",
    "Phu Tho", "Phu Yen", "Quang Binh", "Quang Nam", "Quang Ngai",
    "Quang Ninh", "Quang Tri",
    "Soc Trang", "Son La", "Tay Ninh", "Thai Binh", "Thai Nguyen",
    "Thanh Hoa", "Thua Thien Hue", "Tien Giang", "Tra Vinh",
    "Tuyen Quang", "Vinh Long", "Vinh Phuc",
    "Yen Bai"
]

def normalize_text(s):
    s = s.lower().strip()
    s = unicodedata.normalize('NFD', s)
    return ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')

def suggest_locations(text):
    text_norm = normalize_text(text)
    province_norm_map = {normalize_text(p): p for p in VIETNAM_PROVINCES}
    matches = get_close_matches(text_norm, province_norm_map.keys(), n=5, cutoff=0.4)
    return [province_norm_map[m] for m in matches]


# 2. GEOLOCATION API (OpenWeather)
def get_coordinates(location_name):
    """Chuyển tên tỉnh -> lat, lon có fuzzy fix."""
    name = location_name.strip()
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={name}&limit=5&appid={API_KEY}"
    r = requests.get(url).json()

    if not r:
        # Nếu lỗi -> thử fuzzy suggestion
        suggestions = suggest_locations(name)
        if not suggestions:
            return None, None

        fixed = suggestions[0]
        url2 = f"http://api.openweathermap.org/geo/1.0/direct?q={fixed}&limit=5&appid={API_KEY}"
        r2 = requests.get(url2).json()
        if not r2:
            return None, None
        return r2[0]["lat"], r2[0]["lon"]

    return r[0]["lat"], r[0]["lon"]


# 3. RAINFALL API (PRECIPITATION TILE)
import requests
API_KEY_WAPI = "d243e111150c4739a80135517252711"

def get_rain_forecast(lat, lon, api_key):
    """
    Trả về 5 giá trị:
    - rain_now (mm/h)
    - rain_1h
    - rain_3h
    - rain_6h
    - rain_12h
    Tất cả đều từ WeatherAPI forecast, 100% theo thời gian thật.
    """

    url = (
        f"https://api.weatherapi.com/v1/forecast.json"
        f"?key={API_KEY_WAPI}&q={lat},{lon}&hours=12"
    )

    try:
        js = requests.get(url).json()
    except Exception as e:
        print("Error calling WeatherAPI:", e)
        return (0.0, 0.0, 0.0, 0.0, 0.0)

    # 1) Mưa hiện tại
    rain_now = js.get("current", {}).get("precip_mm", 0.0)

    # 2) List giờ tiếp theo
    hours = js.get("forecast", {}).get("forecastday", [{}])[0].get("hour", [])

    # Đảm bảo đủ 12 giờ
    def safe_precip(index):
        if index < len(hours):
            return hours[index].get("precip_mm", 0.0)
        return 0.0

    rain_1h  = safe_precip(1)   # giờ thứ 1
    rain_3h  = safe_precip(3)
    rain_6h  = safe_precip(6)
    rain_12h = safe_precip(12)

    return (
        float(rain_now),
        float(rain_1h),
        float(rain_3h),
        float(rain_6h),
        float(rain_12h)
    )

# STREAMLIT UI
st.set_page_config(page_title="AI Cảnh Báo Ngập Lụt", layout="wide")
st.title("🌧 AI Cảnh Báo Ngập Lụt — Realtime Weather + Segmentation")

st.write("Dự báo ngập tương lai dựa trên ảnh, segmentation và dữ liệu mưa thời gian thực.")


# LOCATION INPUT
st.subheader("📍 Chọn vị trí lấy dữ liệu thời tiết")

mode = st.radio("Chọn cách nhập:", ["Nhập tên tỉnh (gợi ý tự động)", "Nhập tọa độ"])

lat, lon = None, None

if mode == "Nhập tên tỉnh (gợi ý tự động)":
    user_input = st.text_input("Nhập tên tỉnh/thành phố (VD: Quy Nhon, Ho Chi Minh City):")

    if user_input:
        suggestions = suggest_locations(user_input)
        if len(suggestions) == 0:
            st.error("❌ Không tìm thấy địa điểm phù hợp.")
        else:
            chosen = st.selectbox("Gợi ý:", suggestions)
            lat, lon = get_coordinates(chosen)
            if lat:
                st.success(f"📌 Tọa độ: {lat}, {lon}")

else:
    lat = st.number_input("Latitude:", format="%.6f")
    lon = st.number_input("Longitude:", format="%.6f")

if lat is None or lon is None:
    st.stop()


# WEATHER: RADAR RAIN
st.subheader("🌧 Lượng mưa hiện tại:")
rain_now, rain_1h, rain_3h, rain_6h, rain_12h = get_rain_forecast(lat, lon, API_KEY_WAPI)
st.metric("Rain Radar (mm/h)", f"{rain_now:.2f}")
st.subheader("🌧 Lượng mưa dự báo:")
st.write(f"Trong 1 giờ: {rain_1h:.2f} mm/h")
st.write(f"Trong 3 giờ: {rain_3h:.2f} mm/h")
st.write(f"Trong 6 giờ: {rain_6h:.2f} mm/h")
st.write(f"Trong 12 giờ: {rain_12h:.2f} mm/h")

# IMAGE UPLOAD
uploaded = st.file_uploader("📸 Tải ảnh để phân tích ngập", type=["jpg","jpeg","png"])

if uploaded:
    img = Image.open(uploaded).convert("RGB")

    with NamedTemporaryFile(delete=False, suffix=".png") as tmp:
        img.save(tmp.name)

        # segmentation
        seg_result = predict_flood_level(tmp.name)

        # feature extraction
        ratios, enc = extract_features(tmp.name)

    # UI hiển thị segmentation
    seg_img = seg_result["segmentation"]
    water_ratio = seg_result["water_ratio"]
    level_now = seg_result["flood_text"]

    colA, colB = st.columns(2)
    with colA:
        st.image(img, caption="Ảnh gốc", use_container_width=True)
    with colB:
        st.image(seg_img, caption=f"Segmentation — Ngập: {water_ratio:.1%}", use_container_width=True)

    st.write(f"### 🌊 Mức ngập hiện tại: **{level_now}**")

    semantic = np.array([
        ratios["water_ratio"],
        ratios["building_flooded_ratio"],
        ratios["road_flooded_ratio"],
        ratios["overall_flood_ratio"],
        ratios["tree_ratio"],
        ratios["vehicle_ratio"],
        ratios["pool_ratio"],
        ratios["grass_ratio"],
        rain_now,
        rain_1h,
        rain_3h,
        rain_6h,
        rain_12h,
    ])

    X_input = np.concatenate([semantic, enc])[None, :]
    future_vec = predictor_model.predict(X_input)[0]  # [flood_1h, flood_3h, flood_6h, flood_12h]

    st.subheader("⏱ Chọn mốc thời gian dự báo")
    option = st.radio(
        "Mốc dự báo:",
        ["1 giờ", "3 giờ", "6 giờ", "12 giờ"],
        horizontal=True
    )

    index_map = {"1 giờ": 0, "3 giờ": 1, "6 giờ": 2, "12 giờ": 3}
    future_ratio = float(future_vec[index_map[option]])

    if future_ratio < 0.05:
        future_text = "Không ngập"
    elif future_ratio < 0.15:
        future_text = "Ngập nhẹ"
    elif future_ratio < 0.35:
        future_text = "Ngập vừa"
    else:
        future_text = "Ngập nặng"

    st.metric(f"Mức ngập sau {option}: ", future_text, f"{future_ratio*100:.1f}%")

    if future_ratio >= 0.35:
        st.error("⚠ Cảnh báo ngập nặng!")
    elif future_ratio >= 0.15:
        st.warning("⚠ Nguy cơ ngập — nên theo dõi.")
    else:
        st.success("✔ An toàn — Nguy cơ thấp.")

## Tải Cloudflared (1 lần sau khi kết nối)

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

!cloudflared --version

## Run Streamlit

In [ ]:
%cd /content/drive/MyDrive/CODE/KHKT
!streamlit run app.py & cloudflared tunnel --url http://localhost:8501